# Regex

In [2]:
import re
from bs4 import BeautifulSoup
import requests

In [3]:
data_re = open("mock_data.csv","r")
file = data_re.read()
file_temp = file.split("\n")
file_no_header = ""
for i in file_temp[1:]:
    file_no_header = file_no_header+"\n"+i
data_re.close()

In [4]:
file.split("\n")[0]

'birthday,phone,names,lat_long,email,zip,city'

### 1 To transform the column birthday from European to US data format while leaving the rest as is.

In [5]:
file_1 = re.sub(r"([0-9]{2})\.([0-9]{2})\.([0-9]{4})",r"\3-\2-\1",file_no_header)
print(file_1[:300])


2020-07-05,1-560-294-4480,Leslie H. Howard,"-80.7931, 157.73725",magna.Nam@Praesentinterdumligula.com,05-449,Campina Grande
2019-11-06,1-616-403-7121,Kyra T. Wynn,"-54.65661, 87.93458",amet.ornare@uterosnon.net,1975,Bagh
2020-11-23,1-258-160-9496,Darius G. Huff,"-56.23283, -116.1583",ipsum@erat.co.


### 2 To strip everything BUT the email column.

In [6]:
file_2 = re.sub(r".+,.+,.+,.+,([a-zA-Z0-9\._]+@[a-zA-Z0-9]+\..*),.+,.+",r"\1",file_no_header)
print(file_2[:200])


magna.Nam@Praesentinterdumligula.com
amet.ornare@uterosnon.net
ipsum@erat.co.uk
nunc.est.mollis@auctor.com
neque.venenatis@Phasellus.ca
Proin.mi@a.co.uk
gravida.non.sollicitudin@odioNam.net
Cras@dict


### 3 - To convert all rows to "name [TAB] birthday" only (and strip the rest).  The birthday column should be in the US format. ([TAB]s will allow you to copy and pass its result into excel.)

In [7]:
file_3 = re.sub(r"([0-9]{2})\.([0-9]{2})\.([0-9]{4}),.+,([a-zA-Z]+\s[A-Z]{1}\.\s[a-zA-Z]+),.+,.+,.+,.+",r"\3-\2-\1\t\4",file_no_header)
print(file_3[:200])



2020-07-05	Leslie H. Howard
2019-11-06	Kyra T. Wynn
2020-11-23	Darius G. Huff
2020-05-07	Flynn M. Rodriguez
2020-09-25	Damon K. Potts
2019-12-30	Odessa U. Stewart
2019-03-04	Ruby M. Noble
2020-12-17	


### 4 - To strip everything BUT lat_long AND reorder its entries to be "long [TAB] lat".  ([TAB]s will allow you to copy and pass its result into excel.)

In [8]:
file_4 = re.sub(r".+,.+,.+,\"(.+),\s(.+)\",.+,.+,.+",r"\2\t\1",file_no_header)
print(file_4[:100])


157.73725	-80.7931
87.93458	-54.65661
-116.1583	-56.23283
-93.02564	50.02477
91.87029	34.94064
96.3


# Web Scrapping

In [19]:
# use agents to get access
agent = {'User-Agent': 'Mozilla/5.0'}
page = requests.get('https://www.usnews.com/',headers=agent)
web_content = page.content
soup = BeautifulSoup(web_content,'html.parser')

In [21]:
# Find top Stories

top_header = soup.find(string=re.compile("Top Stories"))
top_content = top_header.parent.parent

In [38]:
type(top_header)

bs4.element.NavigableString

In [37]:
top_header.parent.parent.h3.a.string

'Trump Trial Adjourns Before Crucial Vote'

In [11]:
# Read and print the URL of the _second_ current top story to the screen
links=top_content.h3.a['href']
top_h3=top_content.findAll('h3')
link_list=[]
for tag in top_h3:
    for link in tag.findAll('a', href=True):
        link_list.append(link['href'])

print(link_list[1])

https://www.usnews.com/news/world-report/articles/2020-02-03/china-accuses-us-of-spreading-fear-as-coronavirus-death-toll-rises


In [12]:
# Navigate to the second top story‘s url
top_second = requests.get(link_list[1],headers=agent)

In [13]:
# Read and print the header
soup_2 = BeautifulSoup(top_second.content,'html.parser')
header_second = soup_2.findAll('h1')
print(header_second[0].text)

China Accuses U.S. of Spreading Fear as Coronavirus Death Toll Rises 


In [14]:
# Read and print the header & the first 3 sentences
main_body = soup_2.find('main')
main_content = main_body.find('div',id='usn-toc-content')
body = main_content.findAll('p')

three_list = []
for p in body:
    three_list.append(p.text)
print(header_second[0].text,"\n\n",three_list[0],"\n\n",three_list[1],"\n\n",three_list[2])

China Accuses U.S. of Spreading Fear as Coronavirus Death Toll Rises  

 Evacuees board an evacuation flight for EU nationals, Feb. 2, 2020, at Wuhan Tianhe International Airport in Wuhan in central China's Hubei Province. (Arek Rataj/AP-File) 

 The U.S. is overreacting and spreading fear about the coronavirus outbreak, China's Foreign Ministry said Monday. 

 SEE: 
